In [4]:
import sqlite3
import langchain
import langchain_community
from langchain_community.utilities import SQLDatabase
import os
import langchain_core

In [5]:
from Flemmings_Library.variables import db_path, db
from Flemmings_Library.ollama import get_ollama_model_names

!ollama list > nul 2>&1 # Für Windows: Ollama starten, Ausgabe der Modelle unterdrücken

In [6]:
model_names = get_ollama_model_names(debug=False)
db_dialect = db.dialect
model_name = model_names[0] # Modell festlegen

In [7]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=model_name,
    temperature=0,
)
print(f'{model_name} ausgewählt und geladen.')

llama3.2-vision:11b ausgewählt und geladen.


Schema der Datenbank extrahieren

In [15]:
from Flemmings_Library.extract import extract_schema

schema = extract_schema(db_path, sample_rows=1, verbose=False)
schema_info_str = str(schema)

State-Klasse definieren

In [12]:
from langchain.chains import LLMChain
from typing_extensions import TypedDict
import re

class State(TypedDict):
    schema_info: str       # Loaded information about the DB schema
    prompt: str
    question: str          # Original natural language question
    db_dialect: str        # SQL dialect (e.g., "sqlite")
    parsed_question: str   # (Optional) Parsed or analyzed question
    query: str             # Generated SQL query
    execution_result: str  # Result from executing the SQL query
    answer: str            # Final answer formatted for the user
    error: str             # Error message (if any)

Prompt-Template erstellen

In [13]:
question = "In which city is the customer with the highest customer ID located?"

In [14]:
from Flemmings_Library.variables import chat_prompt_template_sql

In [16]:
formatted_prompt = chat_prompt_template_sql.format(
    schema_info=schema_info_str,
    question=question,
    db_dialect=db_dialect
)
print(formatted_prompt)


System: You are a helpful SQL assistant that provides only SELECT-Statements. Based on the following database schema, translate a natural language query into an SQL query that is executable in my sqlite-Database.

Database Schema:
{'Album': {'columns': ['AlbumId INTEGER (PK)', 'Title NVARCHAR(160)', 'ArtistId INTEGER'], 'foreign_keys': ['ArtistId -> Artist.ArtistId'], 'sample_data': [(1, 'For Those About To Rock We Salute You', 1)]}, 'Artist': {'columns': ['ArtistId INTEGER (PK)', 'Name NVARCHAR(120)'], 'foreign_keys': [], 'sample_data': [(1, 'AC/DC')]}, 'Customer': {'columns': ['CustomerId INTEGER (PK)', 'FirstName NVARCHAR(40)', 'LastName NVARCHAR(20)', 'Company NVARCHAR(80)', 'Address NVARCHAR(70)', 'City NVARCHAR(40)', 'State NVARCHAR(40)', 'Country NVARCHAR(40)', 'PostalCode NVARCHAR(10)', 'Phone NVARCHAR(24)', 'Fax NVARCHAR(24)', 'Email NVARCHAR(60)', 'SupportRepId INTEGER'], 'foreign_keys': ['SupportRepId -> Employee.EmployeeId'], 'sample_data': [(1, 'Luís', 'Gonçalves', 'Embrae

In [17]:
# initialen State festlegen
state: State = {
    "schema_info": schema_info_str,
    "question": question,
    "db_dialect": db.dialect,
    "prompt": formatted_prompt,
    "parsed_question": "",     
    "query": "",
    "execution_result": "",
    "answer": "",
    "error": ""
}

In [18]:
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt_template_sql | llm | StrOutputParser()

In [19]:
llm_output = chain.invoke({
            "schema_info": schema_info_str,
            "question": question,
            "db_dialect": db.dialect
        })